**68_Path**

In [1]:
import os ; import pathlib as pl ; import flopy ; import matplotlib.pyplot as plt ; import numpy as np; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl ; import pandas as pd  
from modflow_devtools.misc import get_env, timed ; write=get_env("WRITE", True);run=get_env("RUN", True) ; from flopy.mf6 import MFSimulation
plot = get_env("PLOT", True) ; plot_show = get_env("PLOT_SHOW", True)

In [2]:
sim_ws  = Path("./68_Path_01")                       ; sim_ws.mkdir(exist_ok=True)       ; sim_name = "ex-prt-mp7-p01"
ex1     = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe'    ; workspace = figs_path = sim_ws    ; output_folder = sim_ws   

In [3]:
length_units = "feet" ; time_units = "days" 
nper = 1 ;nlay = 3 ;nrow = 21 ;ncol = 20 ;delr = 500 ;delc = 500 ;top = 400 ;botm_str = "220.0,200.0,0.0" ;porosity = 0.1 ;rch = 0.005 
kh_str = "50.0,0.01,200.0" ;kv_str = "10.0,0.01,20.0" ;wel_q = -150000 ;riv_h = 320 ;riv_z = 317 ;riv_c = 100000 ;nstp = 1 ;perlen = 1 ;tsmult = 1 
botm = [float(value) for value in botm_str.split(",")]   
kh = [float(value) for value in kh_str.split(",")]  ; kv = [float(value) for value in kv_str.split(",")]

In [4]:
wel_loc = (2, 10, 9)  ; wd = [(wel_loc, wel_q)]     ; riv_iface = 6  ; riv_iflowface = -1  ; rd = []
for i in range(nrow):  rd.append([(0, i, ncol - 1), riv_h, riv_c, riv_z, riv_iface, riv_iflowface]) ; rch_iface = 6 ; rch_iflowface = -1
mp7_particle_data_1a = flopy.modpath.ParticleData([(0, i, 2) for i in range(nrow)],drape=0,
                                                  structured=True,particleids=[i for i in range(nrow)],localz=1.0)
mp7_particle_data_1b = \
flopy.modpath.LRCParticleData(subdivisiondata= \
    [flopy.modpath.FaceDataType(drape=0,verticaldivisions1=0,horizontaldivisions1=0,verticaldivisions2=0,horizontaldivisions2=0,
                                verticaldivisions3=0,horizontaldivisions3=0,verticaldivisions4=0,horizontaldivisions4=0,rowdivisions5=0,
                                columndivisions5=0,rowdivisions6=3,columndivisions6=3,)],lrcregions=[[[0, 0, 0, 0, nrow - 1, ncol - 1]]])

In [5]:
gwf_name = sim_name.replace("ex-prt-", "") + "-gwf"  ; gwf_ws = sim_ws / "gwf"                    ; headfile = f"{gwf_name}.hds"
prt_name = sim_name.replace("ex-prt-", "") + "-prt"  ; prt_ws = sim_ws / "prt"                    ; budgetfile = f"{gwf_name}.cbb"
mp7_name = sim_name.replace("ex-prt-", "") + "-mp7"  ; mp7_ws = sim_ws / "mp7"                    ; budgetfile_prt = f"{prt_name}.cbb"
trackfile_prt = f"{prt_name}.trk"                    ; trackhdrfile_prt = f"{prt_name}.trk.hdr"   ; trackcsvfile_prt = f"{prt_name}.trk.csv"
nodes = {}              ; k,i,j=wel_loc  ; nodes["well"] = [ncol*(nrow *k+i)+j]     ; nodes["river"] = []
for rivspec in rd:
    k, i, j = rivspec[0]                          ; node =  ncol*(nrow *k+i)+j      ; nodes["river"].append(node)

In [14]:
def build_models(example_name):
    gwfsim = flopy.mf6.MFSimulation      (sim_name = gwf_name, exe_name="mf6", version="mf6", sim_ws=gwf_ws) # ____________________________
    flopy.mf6.ModflowTdis    (gwfsim ,time_units = "DAYS" ,nper = nper ,perioddata = [(perlen ,nstp ,tsmult)])
    gwf = flopy.mf6.ModflowGwf           (gwfsim ,modelname = gwf_name ,model_nam_file = f"{gwf_name}.nam" ,save_flows = True)  # _________
    flopy.mf6.ModflowGwfdis  (gwf ,nlay = nlay ,nrow = nrow ,ncol = ncol ,length_units = "FEET" ,delr = delr ,delc = delc ,top = top ,botm = botm)
    flopy.mf6.ModflowGwfic   (gwf ,strt = top)
    flopy.mf6.ModflowGwfnpf  (gwf ,icelltype = [1 ,0 ,0] ,k = kh ,k33 = kv ,save_saturation = True ,save_specific_discharge = True)
    flopy.mf6.ModflowGwfrcha (gwf ,recharge = rch ,auxiliary = ["iface" ,"iflowface"] ,aux = [rch_iface ,rch_iflowface])
    flopy.mf6.ModflowGwfwel  (gwf ,maxbound = 1 ,stress_period_data = {0:wd})
    flopy.mf6.ModflowGwfriv  (gwf ,auxiliary = ["iface" ,"iflowface"] ,stress_period_data = {0:rd})
    flopy.mf6.ModflowGwfoc   (gwf ,head_filerecord = [headfile] , budget_filerecord = [budgetfile], saverecord=[("HEAD","ALL") ,("BUDGET","ALL")])
    flopy.mf6.ModflowIms     (gwfsim ,complexity = "SIMPLE" ,outer_dvclose = 1e-6 ,inner_dvclose = 1e-6 ,rcloserecord = 1e-6)
    prtsim = flopy.mf6.MFSimulation     (sim_name = prt_name ,exe_name = "mf6" ,version = "mf6" ,sim_ws = prt_ws) # _______________________
    flopy.mf6.ModflowTdis (prtsim ,time_units = "DAYS" ,nper = nper ,perioddata = [(perlen ,nstp ,tsmult)])
    prt = flopy.mf6.ModflowPrt          (prtsim ,modelname = prt_name ,model_nam_file = f"{prt_name}.nam")      # _________________________
    flopy.mf6.ModflowGwfdis (prt ,nlay = nlay ,nrow = nrow ,ncol = ncol ,length_units = "FEET" ,delr = delr ,delc = delc ,top = top ,botm = botm)

    izone = np.zeros((nlay, nrow, ncol), dtype=int)
    for l, r, c in gwf.modelgrid.get_lrc(nodes["well"]):  izone[l, r, c] = 1
    for l, r, c in gwf.modelgrid.get_lrc(nodes["river"]): izone[l, r, c] = 2
        
    flopy.mf6.ModflowPrtmip    (prt, pname="mip", porosity=porosity, izone=izone)
    releasepts_1a = list(mp7_particle_data_1a.to_prp(prt.modelgrid))            ; releasepts_1b = list(mp7_particle_data_1b.to_prp(prt.modelgrid))
    flopy.mf6.ModflowPrtprp    (prt ,pname = "prp1a" ,filename = f"{prt_name}_1a.prp" ,nreleasepts = len(releasepts_1a) ,
                                packagedata = releasepts_1a ,perioddata = {0:["FIRST"]} ,exit_solve_tolerance = 1e-5 ,extend_tracking = True)
    flopy.mf6.ModflowPrtprp    (prt ,pname = "prp1b" ,filename = f"{prt_name}_1b.prp" ,nreleasepts = len(releasepts_1b) ,
                                packagedata = releasepts_1b ,perioddata = {0:["FIRST"]} ,exit_solve_tolerance = 1e-10 ,extend_tracking = True)
    
    tracktimes = list(range(0, 72000, 1000))
    
    flopy.mf6.ModflowPrtoc(prt,pname="oc",budget_filerecord=[budgetfile_prt],track_filerecord=[trackfile_prt],trackcsv_filerecord=[trackcsvfile_prt],
                           ntracktimes=len(tracktimes),tracktimes=[(t,) for t in tracktimes],saverecord=[("BUDGET", "ALL")])
    pd = [("GWFHEAD", pl.Path(f"../{gwf_ws.name}/{headfile}")),("GWFBUDGET", pl.Path(f"../{gwf_ws.name}/{budgetfile}"))]
    flopy.mf6.ModflowPrtfmi (prt, packagedata=pd)
    ems = flopy.mf6.ModflowEms  (prtsim ,pname = "ems" ,filename = f"{prt_name}.ems")
    prtsim.register_solution_package (ems ,[prt.name])

    
    mp7 = flopy.modpath.Modpath7 (modelname = mp7_name ,flowmodel = gwf ,exe_name = "mp7" ,model_ws = mp7_ws ,
                                  budgetfilename = budgetfile ,headfilename = headfile)
    flopy.modpath.Modpath7Bas   (mp7 ,porosity = porosity ,defaultiface = {"RCH":6 ,"EVT":6})
    pg1a = flopy.modpath.ParticleGroup            (particlegroupname = "PG1A" ,particledata = mp7_particle_data_1a ,filename = sim_name+"a.sloc")
    pg1b = flopy.modpath.ParticleGroupLRCTemplate (particlegroupname = "PG1B" ,particledata = mp7_particle_data_1b ,filename = sim_name+"b.sloc")
    mpsim = flopy.modpath.Modpath7Sim (mp7 ,simulationtype = "combined" ,trackingdirection = "forward" ,weaksinkoption = "pass_through" ,
                                       weaksourceoption = "pass_through" ,budgetoutputoption = "summary" ,referencetime = [0 ,0 ,0.0] ,
                                       stoptimeoption = "extend" ,timepointdata = [500 ,1000.0] ,zonedataoption = "on" ,zones = izone ,
                                       particlegroups = [pg1a ,pg1b])
    return gwfsim, prtsim, mp7

def write_models(*sims, silent=False):
    for sim in sims:
        if isinstance(sim, MFSimulation): sim.write_simulation(silent=silent)
        else: sim.write_input()
@timed
def run_models(*sims, silent=False):
    for sim in sims:
        if isinstance(sim, MFSimulation): success, buff = sim.run_simulation(silent=silent, report=True)
        else: success, buff = sim.run_model(silent=silent, report=True) ; assert success, pformat(buff)

def scenario(silent=False):
    gwfsim, prtsim, mp7 = build_models(sim_name)
    if write: write_models(gwfsim, prtsim, mp7, silent=silent)
    if run:   run_models(gwfsim, prtsim, mp7, silent=silent)
scenario(silent=True)

run_models took 3218.09 ms


### **Then ...**